# Senior Cubers Worldwide - Weekly Competition

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/scw-comp/

## Initialisation

Basic approach to determine the project directory

In [1]:
import os, sys

projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

## Supported Events

Supported events

In [2]:
sheetMap = \
{
    '3x3x3': '333',
    '2x2x2': '222',
    'oh': '333oh',
    'mega':  'minx',

    '4x4x4': '444',
    '5x5x5': '555',
    '6x6x6': '666',
    '7x7x7': '777',
    
    'pyra': 'pyram',
    'skewb': 'skewb',
    'sq-1': 'sq1',
    'clock': 'clock',

    '3bld': '333bf',
    '4bld': '444bf',
    '5bld': '555bf',
    'fmc': '333fm'
}

responseMap = \
{
    '3x3x3': '333',
    '2x2x2': '222',
    '3x3x3 One-Handed': '333oh',
    'Megaminx': 'minx',

    '4x4x4': '444',
    '5x5x5': '555',
    '6x6x6': '666',
    '7x7x7': '777',

    'Pyraminx': 'pyram',
    'Skewb': 'skewb',
    'Square-1': 'sq1',
    'Clock': 'clock',

    '3x3x3 Blindfolded': '333bf',
    '4x4x4 Blindfolded': '444bf',
    '5x5x5 Blindfolded': '555bf',
    '3x3x3 Fewest Moves': '333fm'
}

## Formatting Functions

Functions to convert results to and from seconds or display an age category

In [3]:
import re

resultPattern = re.compile('^([0-5]?[0-9]:[0-5]|[0-5])?[0-9]\.[0-9][0-9]$|^DNF$|^DNS$')

def numSeconds(value):
    '''Convert float or string to number of seconds - e.g. 1:05.31 returns 65.31'''   
    
    # Numeric result (e.g. SS.cc)
    if isinstance(value, float):
        return round(value, 2)

    # String result (e.g. MM:SS.cc, SS.cc, DNF or DNS)
    elif isinstance(value, str):
        # Some people may have used commas instead of dots
        value = value.replace(',', '.')
        
        # Check that the result is either a time, DNF or DNS
        if not resultPattern.match(value):
            raise ValueError(value)
        
        # Interpret time (e.g. MM:SS.cc or SS.cc), DNF or DNS
        if ':' in value:
            parts = value.split(':')
            return round(int(parts[0]) * 60 + float(parts[1]), 2)
        elif '.' in value:
            return round(float(value), 2)
        elif 'dnf' in value.lower():
            return -1
        elif 'dns' in value.lower():
            return -2
        else:
            raise ValueError(value)

    # This would be very unexpected!
    else:
        raise ValueError(value)


def formatResult(value, event):
    '''Convert number of seconds to displayable time - e.g. 65.31 returns 1:05.31'''
    
    if value is not None:
        if value > 0:
            if event == '333fm':
                return '{:d}'.format(int(value))
            else:
                if value > 60:
                    return '{:d}:{:05.2f}'.format(int(value // 60), value - int(value // 60) * 60)
                else:
                    return '{:.2f}'.format(value)
        else:
            if value == -1:
                return 'DNF'
            elif value == -2:
                return 'DNS'
            else:
                return '?'

    # This would be very unexpected!
    else:
        raise ValueError(value)

In [4]:
def interpretAge(age):
    '''Interpret age, however it is written and return an integer'''
    
    age = age.lower()
    if 'under' in age:
        age = int(age.replace('under', '').replace(' ', '')) - 10
    elif 'over' in age:
        age = int(age.replace('over', '').replace(' ', ''))
    elif '<' in age:
        age = int(age.replace('<', '').replace(' ', '')) - 10
    elif '>' in age:
        age = int(age.replace('>', '').replace(' ', ''))
    else:
        age = int(age.replace('+', ''))
        
    return age

    
def formatAge(age):
    '''Format age for report'''
    
    if age < 40:
        return '<{}'.format(age + 10)
    else:
        return '{}+'.format(age)

In [5]:
def formatFacebookLink(link):
    '''Change mobile links to regular Facebook link'''
    
    if link:
        if '//m.' in link:
            link = link.replace('//m.', '//www.')
        if '?view=permalink&id=' in link:
            link = link.replace('?view=permalink&id=', '/permalink/')
        if '?' in link:
            link = link[:link.find('?')]
        if not link.endswith('/'):
            link = link + '/'

    return link

## Calculation Functions

Functions to calculate averages, etc

In [6]:
secsInDay = 86400

def calculateBest(solves):
    '''Calculate best result from list of solves'''

    best = -1
    for solve in solves:
        if solve > 0 and (best < 0 or solve < best):
            best = solve

    return best


def calculateAverage(solves):
    '''Calculate Mo3 / Ao5 from list of solves'''

    average = -1
    
    # Copy list of solves prior to manipulation
    tmpSolves = solves.copy()

    # Remove DNS - e.g. doing Mo3 rather than Ao5
    while -2 in tmpSolves:
        tmpSolves.remove(-2)

    # Convert DNF to 1 day
    for i in range(len(tmpSolves)):
        if tmpSolves[i] < 0:
            tmpSolves[i] = secsInDay

    # Sort solves
    tmpSolves = sorted(tmpSolves)

    # Calculate Ao5
    if (len(tmpSolves)) >= 4:
        if tmpSolves[3] == secsInDay:
            average = -1
        else:
            average = round((tmpSolves[1] + tmpSolves[2] + tmpSolves[3]) / 3, 2)

    # Calculate Mo3
    elif (len(tmpSolves)) == 3:
        if tmpSolves[2] == secsInDay:
            average = -1
        else:
            average = round((tmpSolves[0] + tmpSolves[1] + tmpSolves[2]) / 3, 2)

    return average

## Generic Class

Generic class to ensure that all custom classes are printable

In [7]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

## Spreadsheet Classes

Class to parse result spreadsheets

In [8]:
from xlrd import open_workbook
import csv

class Spreadsheet(Printable):
    def __init__(self, filename):
        self.filename = filename
        self.date = os.path.basename(os.path.dirname(filename))
        self.workbook = open_workbook(filename)

In [9]:
class LegacySheet(Spreadsheet):
    def processSheet(self, sheet):
        '''Process a sheet which was filled in by participants'''

        event = sheetMap[sheet.name.lower()]
        columnNames = []
        rows = []
        rowsIdx = []

        for rowNo in range(sheet.nrows):

            # Process header row
            if rowNo == 2:
                for colNo in range(sheet.ncols):
                    value = sheet.cell(rowNo, colNo).value.strip()
                    
                    # Simple string matches
                    if value in ('Name', 'Age', 'Result', 'Best'):
                        columnNames.append(value)
                    elif value.startswith('Solve'):
                        columnNames.append(value)
                        
                    # Simple substring matches
                    elif 'Ao5' in value or 'Mo3' in value:
                        columnNames.append('Average')
                    elif 'link' in value.lower():
                        columnNames.append('Link')
                    elif value.startswith('Comment'):
                        columnNames.append('Comment')
                        
                    # Columns to ignore
                    elif value.startswith('Pos') or value.startswith('Award') or value == '':
                        columnNames.append(None)

                    # Report any other columns
                    else:
                        print('WARNING: Unexpected field "{}" in {} ({})'.format(value, sheet.name, self.date))

                row = []
                for columnName in columnNames:
                    if columnName:
                        row.append(columnName)
                rows.append(row)

            # Process result row - must be after header on row 3
            elif rowNo >= 3:
                name = None
                age = None
                best = None
                average = None
                row = []
                solves = []
                
                for colNo in range(sheet.ncols):
                    if columnNames[colNo]:
                        fieldValue = sheet.cell(rowNo, colNo).value
                        if isinstance(fieldValue, str):
                            fieldValue = fieldValue.strip()
                        
                        # Retain name
                        if columnNames[colNo] == 'Name':
                            name = fieldValue
                          
                        # Standardise age
                        elif columnNames[colNo] == 'Age':
                            try:
                                age = interpretAge(fieldValue)
                                fieldValue = formatAge(age)
                            except:
                                print('ERROR: Age "{}" for {} in {} ({})'.format(fieldValue, name, sheet.name, self.date))
                                raise

                        # Standardise result fields
                        elif columnNames[colNo] == 'Average' or columnNames[colNo] == 'Best' \
                                or columnNames[colNo] == 'Result' or columnNames[colNo].startswith('Solve'):
                            try:
                                # Convert result to time in seconds
                                result = numSeconds(fieldValue)
                                
                                # Add to list of solves and remember best / average
                                if (columnNames[colNo].startswith('Solve')) or columnNames[colNo] == 'Result':
                                    solves.append(result)
                                elif columnNames[colNo] == 'Best':
                                    best = result
                                elif columnNames[colNo] == 'Average':
                                    average = result

                                # Use the formatted result in the output
                                fieldValue = formatResult(result, event)
                            except:
                                print('ERROR: Result "{}" for {} in {} ({})'.format(fieldValue, name, sheet.name, self.date))
                                raise

                        # Standardise Facebook links
                        elif columnNames[colNo] == 'Link':
                            try:
                                fieldValue = formatFacebookLink(fieldValue)
                                if '/permalink/' not in fieldValue and '/videos/' not in fieldValue:
                                    fieldValue = ''
                            except:
                                print('ERROR: Link "{}" for {} in {} ({})'.format(fieldValue, name, sheet.name, self.date))
                                raise

                        row.append(fieldValue)

                # Check best
                if best:
                    if best != calculateBest(solves):
                        print('ERROR: Best incorrect for {} in {} ({}) - calculated {}'.format(
                            name, event, self.date, formatResult(calculateBest(solves), event)))
                else:
                    best = calculateBest(solves)

                # Check average
                if average:
                    diff = round(average - calculateAverage(solves), 2)
                    if diff < -0.01 or diff > 0.01:
                        print('ERROR: Average incorrect for {} in {} ({}) - calculated {}'.format(
                            name, event, self.date, formatResult(calculateAverage(solves), event)))
                else:
                    average = calculateAverage(solves)

                # Record row
                rows.append(row)

                # Create index entry, patching DNS and DNF to 1 day
                if best < 0:
                    best = secsInDay
                if average < 0:
                    average = secsInDay
                rowsIdx.append((rowNo - 2, best, average, 100 - age, name))

        # Sort index
        if event.endswith('bf'):
            rowsIdx = sorted(rowsIdx, key = lambda x: (x[1], x[2], x[3], x[4]))
        else:
            rowsIdx = sorted(rowsIdx, key = lambda x: (x[2], x[1], x[3], x[4]))
        
        # Sort rows
        sortedRows = rows[:1]
        for rowIdx in rowsIdx:
            sortedRows.append(rows[rowIdx[0]])

        # Write CSV
        filename = os.path.join(os.path.dirname(self.filename), event + '.csv')
        with open(filename, 'w') as outfile:
            csvWriter = csv.writer(outfile, quoting = csv.QUOTE_MINIMAL, lineterminator = os.linesep)
            csvWriter.writerows(sortedRows)


    def processSheets(self, latest):
        '''Process a spreadsheet which has been downloaded from Google Sheets'''

        for sheet in self.workbook.sheets():
            if sheet.name.lower() in sheetMap:
                self.processSheet(sheet)

In [10]:
class ResponseSheet(Spreadsheet):

    def processSheet(self, sheet):
        '''Process a sheet which was filled in by Google Forms'''

        columnNames = []
        eventRows = {}
        eventRowsIdx = {}
        eventCols = {}
        eventIdx = -1
        commentIdx = -1
        linkIdx = -1

        for rowNo in range(sheet.nrows):

            # Process header row
            if rowNo == 0:
                for colNo in range(sheet.ncols):
                    value = sheet.cell(rowNo, colNo).value.strip()
                    
                    # Simple column names
                    if value in ('Name', 'Age', 'Result'):
                        columnNames.append(value)
                    elif value.startswith('Solve'):
                        columnNames.append(value)
                        
                    # Variable column names
                    elif value.startswith('WCA ID'):
                        columnNames.append('WCA ID')
                    elif value.startswith('Event'):
                        eventIdx = colNo
                        columnNames.append('Event')
                    elif value in ('Best', 'Single'):
                        columnNames.append('Best')
                    elif 'Ao5' in value or 'Mo3' in value:
                        columnNames.append('Average')
                    elif 'link' in value.lower():
                        linkIdx = colNo
                        columnNames.append('Link')
                    elif value.startswith('Comment'):
                        commentIdx = colNo
                        columnNames.append('Comment')

                    # Report any other columns
                    elif value in ('Timestamp'):
                        columnNames.append(None)
                    else:
                        print('WARNING: Unexpected field "{}" in {} ({})'.format(value, sheet.name, self.date))

            # Process result row
            else:
                name = None
                age = None
                event = None
                best = None
                average = None
                row = []
                solves = []
                cols = []
                
                for colNo in range(sheet.ncols):
                    fieldValue = sheet.cell(rowNo, colNo).value
                    if isinstance(fieldValue, str):
                        fieldValue = fieldValue.strip()
                        
                    if columnNames[colNo]:
                        # Retain name
                        if columnNames[colNo] == 'Name':
                            name = fieldValue

                        # Standardise event
                        elif columnNames[colNo] == 'Event':
                            event = responseMap[fieldValue]
                            fieldValue = None

                        # Standardise age
                        elif columnNames[colNo] == 'Age':
                            try:
                                age = interpretAge(fieldValue)
                                fieldValue = formatAge(age)
                            except:
                                print('ERROR: Age "{}" for {} in {} ({})'.format(fieldValue, name, event, self.date))
                                raise

                        # Standardise result fields
                        elif columnNames[colNo] == 'Average' or columnNames[colNo] == 'Best' \
                                or columnNames[colNo] == 'Result' or columnNames[colNo].startswith('Solve'):
                            try:
                                if fieldValue:
                                    # Convert result to time in seconds
                                    result = numSeconds(fieldValue)

                                    # Add to list of solves and remember best / average
                                    if (columnNames[colNo].startswith('Solve')) or columnNames[colNo] == 'Result':
                                        solves.append(result)
                                    elif columnNames[colNo] == 'Best':
                                        best = result
                                    elif columnNames[colNo] == 'Average':
                                        average = result

                                    # Use the formatted result in the output
                                    fieldValue = formatResult(result, event)
                            except:
                                print('ERROR: Result "{}" for {} in {} ({})'.format(fieldValue, name, event, self.date))
                                raise

                        # Standardise Facebook links
                        elif columnNames[colNo] == 'Link':
                            try:
                                fieldValue = formatFacebookLink(fieldValue)
                                if '/permalink/' not in fieldValue and '/videos/' not in fieldValue:
                                    fieldValue = ''
                            except:
                                print('ERROR: Link "{}" for {} in {} ({})'.format(fieldValue, name, event, self.date))
                                raise
                            
                        if fieldValue:
                            row.append(fieldValue)
                            cols.append(colNo)
                        else:
                            row.append('')
                    else:
                        row.append('')

                # Check best
                if best:
                    if best != calculateBest(solves):
                        print('ERROR: Best incorrect for {} in {} ({}) - calculated {}'.format(
                            name, event, self.date, formatResult(calculateBest(solves), event)))
                else:
                    best = calculateBest(solves)
                    
                # Check average
                if average:
                    diff = round(average - calculateAverage(solves), 2)
                    if diff < -0.01 or diff > 0.01:
                        print('ERROR: Average incorrect for {} in {} ({}) - calculated {}'.format(
                            name, event, self.date, formatResult(calculateAverage(solves), event)))
                else:
                    average = calculateAverage(solves)

                # Report duplicate submission
                if event in eventRows and name in eventRows[event]:
                    print('INFO: Duplicate submission for {} in {} ({})'.format(name, event, self.date))
                    
                    oldRow = eventRows[event][name].copy()
                    del(oldRow[commentIdx])
                    del(oldRow[linkIdx])
                    print('{}'.format(','.join(filter(None, oldRow))))
                    
                    newRow = row.copy()
                    del(newRow[commentIdx])
                    del(newRow[linkIdx])
                    print('{}'.format(','.join(filter(None, newRow))))
                    print()

                # Record columns
                if event not in eventCols:
                    eventCols[event] = cols
                else:
                    for col in cols:
                        if col not in eventCols[event]:
                            eventCols[event].append(col)

                # Record row
                if event not in eventRows:
                    eventRows[event] = {}
                eventRows[event][name] = row

                # Create index entry, patching DNS and DNF to 1 day
                if best < 0:
                    best = secsInDay
                if average < 0:
                    average = secsInDay
                if event not in eventRowsIdx:
                    eventRowsIdx[event] = {}
                eventRowsIdx[event][name] = (rowNo, best, average, 100 - age, name)

        # Tidy up event columns - sort columns numerically then move "link" and comment" to the end
        for event in eventCols:
            eventCols[event] = sorted(eventCols[event])
            if linkIdx in eventCols[event]:
                eventCols[event].remove(linkIdx)
            if commentIdx in eventCols[event]:
                eventCols[event].remove(commentIdx)
            eventCols[event].append(linkIdx)
            eventCols[event].append(commentIdx)

        # Save events
        for event in eventRows:
            rows = []
            rowsIdx = []

            # Sort index
            for name in eventRowsIdx[event]:
                rowsIdx.append(eventRowsIdx[event][name])

            if event.endswith('bf'):
                rowsIdx = sorted(rowsIdx, key = lambda x: (x[1], x[2], x[3], x[4]))
            else:
                rowsIdx = sorted(rowsIdx, key = lambda x: (x[2], x[1], x[3], x[4]))

            # Create header (comment last)
            row = []
            for col in eventCols[event]:
                if col != commentIdx:
                    row.append(columnNames[col])
            row.append(columnNames[commentIdx])
            rows.append(row)

            # Create results (comment last)
            for rowIdx in rowsIdx:
                name = rowIdx[4]
                row = []
                for col in eventCols[event]:
                    row.append(eventRows[event][name][col])
                rows.append(row)
            
            filename = os.path.join(os.path.dirname(self.filename), event + '.csv')
            with open(filename, 'w') as outfile:
                csvWriter = csv.writer(outfile, quoting = csv.QUOTE_MINIMAL, lineterminator = os.linesep)
                csvWriter.writerows(rows)


    def processSheets(self, latest):
        '''Process a spreadsheet which has been downloaded from Google Sheets'''

        for sheet in self.workbook.sheets():
            self.processSheet(sheet)

## Main Code

Process all competitions

In [11]:
import glob
import time

pc1 = time.perf_counter()

In [12]:
# Process spreadsheets

datePattern = '[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]'
xlsxPattern = '*.xlsx'

persons = {}
competitions = {}

filenames = glob.glob(os.path.join(projdir, 'data', datePattern, xlsxPattern))
refresh = False
latest = False

for filename in filenames:
    if filename == filenames[-1]:
        latest = True

    if refresh or latest:
        if 'Responses' in filename:
            spreadsheet = ResponseSheet(filename)
        else:
            spreadsheet = LegacySheet(filename)

        spreadsheet.processSheets(latest)

INFO: Duplicate submission for Jae Park in 555bf (2020-06-09)
Jae Park,40+,2015PARK24,DNF,14:54.39,DNS,14:54.39,DNF
Jae Park,40+,2015PARK24,DNF,14:54.39,DNF,14:54.39,DNF

INFO: Duplicate submission for Chris Thames in 333 (2020-06-09)
Chris Thames,<40,2019THAM05,30.25,30.58,46.47,26.69,32.77,26.69,31.20
Chris Thames,<40,2019THAM05,30.25,30.58,46.47,26.69,32.77,26.69,31.20

INFO: Duplicate submission for Chris Thames in 333 (2020-06-09)
Chris Thames,<40,2019THAM05,30.25,30.58,46.47,26.69,32.77,26.69,31.20
Chris Thames,<40,2019THAM05,30.25,30.58,46.47,26.69,32.77,26.69,31.20

INFO: Duplicate submission for Chris Thames in 222 (2020-06-09)
Chris Thames,<40,2019THAM05,10.48,13.11,8.52,12.15,17.04,8.52,11.91
Chris Thames,<40,2019THAM05,10.48,13.11,8.52,12.15,17.04,8.52,11.91

INFO: Duplicate submission for Chris Thames in minx (2020-06-09)
Chris Thames,<40,2019THAM05,6:18.01,DNS,DNS,DNS,DNS,6:18.01,DNF
Chris Thames,<40,2019THAM05,6:18.01,DNS,DNS,DNS,DNS,6:18.01,DNF

INFO: Duplicate submissi

In [13]:
pc2 = time.perf_counter()
print("Conversion completed in %0.2f seconds" % (pc2 - pc1))

Conversion completed in 19.18 seconds


## All Done!